In [ ]:
warnings.filterwarnings('ignore') #Apaga las warnings, no hay errores, solo tira un monton de mensajes

In [ ]:
def read_json(k,filename,swp,shp,file_extension,Fs):
    #Lee archivo json o psimesh
    data_all=pd.DataFrame()

    if file_extension=='.psimesh':
        response = open(filename,encoding = 'utf-8')
    else:
        response = open(filename)
    data_read = json.load(response)
   
    #Lee todos los trials. Queda todo junto.
    hoja=0
    are_trials=True
    while are_trials:
        try:
            try:
                data = json_normalize(data_read["stimData"][hoja]["eyetrackerData"])
                data["Trial"]=hoja+1
                data_all=pd.concat([data_all,data],axis=0,ignore_index=True)
                hoja=hoja+1
            except:
                trial = 'trial'+str(hoja+1)
                data = json_normalize(data_read["all"][trial]["data"])
                data["Trial"]=hoja+1
                data_all=pd.concat([data_all,data],axis=0,ignore_index=True)
                hoja=hoja+1
        except:
            are_trials=False
            break
    data=data_all.copy()    

    #Grilla temporal. No es uniforme. Arranca en cero
    t1=np.asarray(data["t"]-data["t"][0])
    if np.all(data["x"]<=1):
        x1=np.asarray(data["x"]*swp)
        y1=np.asarray(data["y"]*shp)
    else:
        x1=np.asarray(data["x"])
        y1=np.asarray(data["y"])
    
    if 'trialScores' in data_read:
        (head,file2)=os.path.split(filename)
        file1=os.path.splitext(file2)
        file_out=os.path.join(directory_out, "trialscores" + "_" + file1[0] + ".txt")
        sourceFile = open(file_out, 'w')
        print(data_read['trialScores'], file = sourceFile)
        sourceFile.close()
    if 'eventLog' in data_read:
        file_out=os.path.join(directory_out, "eventlog" + "_" + file1[0] + ".txt")
        sourceFile = open(file_out, 'w')
        print(data_read['eventLog'], file = sourceFile)
        sourceFile.close()     
  
    #Arregla la duración de los trials del Ret, que el archivo no coincide con lo visto en el PsiMesh.
    #También reescalea a la pantalla correcta (los datos están 'corridos')
    (head,file2)=os.path.split(filename)
    files_xlsx=glob.glob(head + 'Duration_trials.xlsx')
    if len(files_xlsx) !=0:
        time_trials = pd.read_excel (files_xlsx,engine='openpyxl')
        ntrials=data["Trial"][t1.size-1]
        t1p=np.linspace(0,time_trials[ntrials][k],num=t1.size)
        n=0
        trials=0
        while n<t1.size:
            if t1p[n]<=time_trials[trials][k]:
                data["Trial"][n]=trials+1
                n=n+1
            else:
                trials=trials+1
            if trials==ntrials+1:
                break
    
    #Interpola x1 e y1 lineal para la grilla del tiempo uniforme
    #Renormaliza la grilla temporal uniforme a la frecuencia del eye-tracker
    step=int(1000/Fs)
    t2=np.arange(t1[0],t1[t1.size-1],step)
    fin=t2.size
    dfc = np.zeros(fin,dtype={"names":('t','xp','yp','trial','interp'),'formats':('float','float','float','int','int')})
    fx = interp1d(t1, x1, kind='linear',fill_value='extrapolate')
    fy = interp1d(t1, y1, kind='linear',fill_value='extrapolate')
    x2 = fx(t2)
    y2 = fy(t2)
    dfc["t"]=t2
    dfc["xp"]=x2 
    dfc["yp"]=y2 
    
    #Trials originales
    trials_data=data["Trial"]
    indx_trial_data=np.diff(trials_data)
    res=np.array(np.where(indx_trial_data!=0))
    res=np.squeeze(res,0)
    tchange=t1[res]
    tchange=np.insert(tchange,0,0)
    tchange=np.append(tchange,t2[t2.size-1]+1)
    #Corrige los trials con la nueva grilla de tiempo
    i=0
    n=0
    while n<t2.size:
        if t2[n]>=tchange[i] and t2[n]<tchange[i+1]:
            if i<res.size:
                dfc["trial"][n]=data["Trial"][res[i]]
            else:
                dfc["trial"][n]=data["Trial"][t1.size-1]
            n=n+1
        else:
            i=i+1
            if i==res.size+1:
                break
                
    #Indica los puntos donde hay más de 100 ms seguidos interpolados           
    for n in range(0,t1.size-1,1):
        if t1[n+1]-t1[n]>100:
            dfc["interp"][(t1[n]<=t2) & (t2<=t1[n+1])]=-1
    
    #Pasa el (0,0) de las coordenadas al centro de la pantalla
    dfc["xp"]=dfc["xp"]-swp/2
    dfc["yp"]=dfc["yp"]-shp/2       
    
    #Corrige la posición del Ret que no está bien calibrado. Hay que hacerlo después de la transformación al (0,0)
    if len(files_xlsx) !=0:
        xa=time_trials["xa"][k]
        ya=time_trials["ya"][k]
        yfact=time_trials["facty"][k]
        dfc["xp"]=dfc["xp"]-xa
        dfc["yp"]=(dfc["yp"]-ya)*yfact
            
    #Guarda en dataframe        
    df=pd.DataFrame(dfc)
    
    #Cierra archivo .json o .psimesh
    response.close()
    return (df)

In [ ]:
#Acomoda los datos en una matriz, chequea onset y offset de sacadas, saca tamaños y duraciones.
def data_to_frame(df_int,index_onset,index_offset):
    
    w_interp=df_int["vpr"]
    
    t=df_int["t"]
    theta_sz=t.size
    data = np.zeros(theta_sz,dtype={"names":('t','xp','vpx','yp','vpy','event','trial','interp'),'formats':('float','float','float','float','float','int','int','int')})
    data["t"]=t
    data["xp"]=df_int["xp"]
    data["yp"]=df_int["yp"]
    data["trial"]=df_int["trial"]
    data["interp"]=df_int["interp"]
    data["vpx"]=df_int["vpx"]
    data["vpy"]=df_int["vpy"]
    
    #Llena con 0 fijaciones y 1 sacadas. Esto ya fusiona sacadas 'superpuestas'
    nsac=index_onset.size
    data["event"][:]=0
    for j in range(0,nsac,1):
            data["event"][index_onset[j]-1:index_offset[j]]=1
    
    #Busca nuevos Onset y offset de SACADAS
    index_onset=np.array(np.where(np.diff(data["event"])==1))+1
    index_offset=np.array(np.where(np.diff(data["event"])==-1))
    
    index_onset=np.squeeze(index_onset,0)
    index_offset=np.squeeze(index_offset,0)
    
    if data["event"][0]==1:
        index_onset = np.hstack([0, index_onset])
            
    nsac=index_onset.size
    for n in range(1,nsac):
        fix=t[index_onset[n]-1]-t[index_offset[n-1]+1]
        sac=t[index_offset[n]]-t[index_onset[n]]
        #Fusiona sacadas si entre ellas hay una fijación de duración menor o igual a 20 ms
        if fix<=20:
            data["event"][index_offset[n-1]+1:index_onset[n]]=1
            index_onset[n]=index_onset[n-1]
            index_onset[n-1]=-1
            index_offset[n-1]=-1
        #Fusiona fijaciones si entre ellas hay una sacada de duración menor o igual a 20 ms
        if sac<=20:
            data["event"][index_onset[n]:index_offset[n]]=0
            index_onset[n]=-1
            index_offset[n]=-1
        #Borra eventos donde hay Nan (no hay datos)
        if np.any(np.isnan(data["xp"][index_onset[n]:index_offset[n]])):
            index_onset[n]=-1
            index_offset[n]=-1
        
    #print(index_offset)
    data_frame=pd.DataFrame(data)
    
    return (index_onset,index_offset,data_frame)

In [ ]:
#Acomoda los datos en una matriz, chequea onset y offset de sacadas, saca tamaños y duraciones.
def saccades_to_frame(df_int,index_onset,index_offset):
    w_peaks1_sz=index_onset.size
    saccades = np.zeros(w_peaks1_sz,dtype={"names":('number','onset', 'offset','sizep','duration','wpeak','trial'),
                          'formats':('int','int', 'int','float','float','float','int')})
    
    w_interp=df_int["vpr"]
    x=df_int["xp"]
    y=df_int["yp"]
    
    t=df_int["t"]
    dsize=t.size
    n=0
    while n<w_peaks1_sz:
        #Si hubo algún problema de detección de onset u offset se marcó con -1 y se pasa de largo.
        if index_onset[n] == -1:
            saccades["number"][n]=0
            n=n+1
            continue
        if index_offset[n] == -1:
            saccades["number"][n]=0
            n=n+1
            continue
        saccades["number"][n]=n+1
        saccades["onset"][n]=index_onset[n]
        saccades["offset"][n]=index_offset[n]
        saccades["sizep"][n]=np.sqrt((x[index_offset[n]]-x[index_onset[n]])**2+(y[index_offset[n]]-y[index_onset[n]])**2)
        w_peaks, _ = find_peaks(w_interp[saccades["onset"][n]:saccades["offset"][n]])
        if w_peaks.size==0:
            saccades["wpeak"][n]=np.amax(w_interp[saccades["onset"][n]:saccades["offset"][n]])
        else:
            saccades["wpeak"][n]=np.amax(w_interp[w_peaks+saccades["onset"][n]])
        saccades["duration"][n]=np.abs(t[index_offset[n]]-t[index_onset[n]])
        n+=1
       
    saccades1=np.delete(saccades, np.where(saccades["number"] == 0), axis=0)
    nsac=saccades1.size
    saccades1["number"][:]=np.arange(nsac)
    
    #Trials from data
    trials_data=df_int["trial"]
    indx_trial_data=np.diff(trials_data)
    res=np.array(np.where(indx_trial_data!=0))
    res=np.squeeze(res,0)
    ntrials=res.size
    fin_trial=np.zeros(ntrials,dtype='int')
    fin_trial[0:ntrials-1]=res[0:ntrials-1];
    fin_trial[-1]=dsize-1;
    ini_trial=np.zeros(ntrials,dtype='int');
    ini_trial[0]=0;
    ini_trial[1:ntrials]=fin_trial[0:ntrials-1]+1;
    
    trial=0
    n=0
    while trial<ntrials:
        while n<nsac:
            j0=ini_trial[trial]
            jf=fin_trial[trial]
            if saccades1["offset"][n]<=jf and saccades1["onset"][n]>=j0:
                saccades1["trial"][n]=trials_data[saccades1["onset"][n]]
                n=n+1
                continue
            if saccades1["offset"][n]>jf and saccades1["onset"][n]>j0 and saccades1["onset"][n]<=jf:
                saccades1["trial"][n]=trials_data[saccades1["onset"][n]]
                n=n+1
                continue
            if saccades1["offset"][n]>jf and saccades1["onset"][n]>jf: 
                trial=trial+1
                continue
        if n==nsac:
            break
    
    saccades_frame=pd.DataFrame(saccades1)
    
    return (saccades_frame)

In [ ]:
#Acomoda los datos en una matriz, chequea onset y offset de sacadas, saca tamaños y duraciones.
def fix_to_frame(df_int,saccades_frame):
    index_onset=saccades_frame["onset"]
    index_offset=saccades_frame["offset"]
    nsize=index_onset.size
    fix = np.zeros(nsize+1,dtype={"names":('number','onset','offset','duration','avg_loc_x','avg_loc_y','trial'),
                                  'formats':('int','int','int','float','float','float','int')})

    x=df_int["xp"]
    y=df_int["yp"]
    
    t=df_int["t"]
    dsize=t.size
    n=0
    while n<nsize+1:
        fix["number"][n]=n
        if n==0:
            if index_onset[0] !=0:
                fix["onset"][n]=0
                fix["offset"][n]=index_onset[n]-1
                fix["avg_loc_x"][n]=np.mean(x[0:index_onset[n]-1])
                fix["avg_loc_y"][n]=np.mean(y[0:index_onset[n]-1])
                fix["duration"][n]=t[index_onset[n]-1]-t[0]
        if 0<n<nsize:
            fix["onset"][n]=index_offset[n-1]+1
            fix["offset"][n]=index_onset[n]-1
            fix["avg_loc_x"][n]=np.mean(x[index_offset[n-1]+1:index_onset[n]-1])
            fix["avg_loc_y"][n]=np.mean(y[index_offset[n-1]+1:index_onset[n]-1])
            fix["duration"][n]=t[index_onset[n]-1]-t[index_offset[n-1]+1]
        if index_offset[nsize-1] != dsize-1:
            if n==nsize:
                fix["onset"][n]=index_offset[n-1]+1
                fix["offset"][n]=dsize-1
                fix["avg_loc_x"][n]=np.mean(x[index_offset[n-1]+1:dsize-1])
                fix["avg_loc_y"][n]=np.mean(y[index_offset[n-1]+1:dsize-1])
                fix["duration"][n]=t[dsize-1]-t[index_offset[n-1]+1]
        n+=1
            
    #Trials from data
    trials_data=df_int["trial"]
    indx_trial_data=np.diff(trials_data)
    res=np.array(np.where(indx_trial_data!=0))
    res=np.squeeze(res,0)
    ntrials=res.size
    fin_trial=np.zeros(ntrials,dtype='int')
    fin_trial[0:ntrials-1]=res[0:ntrials-1];
    fin_trial[-1]=dsize-1;
    ini_trial=np.zeros(ntrials,dtype='int');
    ini_trial[0]=0;
    ini_trial[1:ntrials]=fin_trial[0:ntrials-1]+1;
    
    trial=0
    n=0
    while trial<ntrials:
        while n<nsize+1:
            j0=ini_trial[trial]
            jf=fin_trial[trial]
            if fix["offset"][n]<=jf and fix["onset"][n]>=j0:
                fix["trial"][n]=trials_data[fix["onset"][n]]
                n=n+1
                continue
            if fix["offset"][n]>jf and fix["onset"][n]>j0 and fix["onset"][n]<=jf:
                fix["trial"][n]=trials_data[fix["onset"][n]]
                n=n+1
                continue
            if fix["offset"][n]>jf and fix["onset"][n]>jf: 
                trial=trial+1
                continue
        if n==nsize+1:
            break
    
    fix_frame=pd.DataFrame(fix)
    
    return (fix_frame)

In [ ]:
#Lee archivo csv
def read_csv(filename,init,fin):
    data = pd.read_csv (filename,decimal=",", delimiter=";")
    #Cambia las comas por puntos, sino no los puede cambiar a float
    data['LEFT_GAZE_X'] = [str(x).replace(',', '.') for x in data['LEFT_GAZE_X']]
    data['LEFT_GAZE_Y'] = [str(x).replace(',', '.') for x in data['LEFT_GAZE_Y']]
    data['RIGHT_GAZE_X'] = [str(x).replace(',', '.') for x in data['RIGHT_GAZE_X']]
    data['RIGHT_GAZE_Y'] = [str(x).replace(',', '.') for x in data['RIGHT_GAZE_Y']]
    #Cambia las columnas objeto a float
    s=pd.Series(data["LEFT_GAZE_X"])
    data["LEFT_GAZE_X"] = pd.to_numeric(s, errors='coerce')
    s=pd.Series(data["LEFT_GAZE_Y"])
    data["LEFT_GAZE_Y"] = pd.to_numeric(s, errors='coerce')
    s=pd.Series(data["RIGHT_GAZE_X"])
    data["RIGHT_GAZE_X"] = pd.to_numeric(s, errors='coerce')
    s=pd.Series(data["RIGHT_GAZE_Y"])
    data["RIGHT_GAZE_Y"] = pd.to_numeric(s, errors='coerce')
    #Cambio el tiempo TIMESTAMP para que empiece de cero (ms)
    data["TIMESTAMP"]=data["TIMESTAMP"]-data["TIMESTAMP"][init]
    #Tomo una parte del experimento para procesar. Guardo tiempo en t,horizontal en x y vertical en y, 
    #promediados en binocular
    t=data ["TIMESTAMP"][init:fin]
    x=(data["LEFT_GAZE_X"][init:fin]+data["RIGHT_GAZE_X"][init:fin])/2
    y=(data["LEFT_GAZE_Y"][init:fin]+data["RIGHT_GAZE_Y"][init:fin])/2
    
    nz=t.size
    dfc = np.zeros(nz,dtype={"names":('t','xp','yp','trial','interp'),'formats':('float','float','float','int','int')})
    dfc["t"]=t
    dfc["xp"]=x
    dfc["yp"]=y
    
    df=pd.DataFrame(dfc)
    
    return(df)

In [ ]:
#Imprime _frame a un archivo csv
def print_to_csv(saccades_frame,fix_frame,data_frame,filename1,filename2,filename3):
    saccades_frame.to_csv (filename1,  header=True, index=False)
    fix_frame.to_csv (filename2,  header=True, index=False)
    data_frame.to_csv (filename3,  header=True, index=True)
    return

In [ ]:
#Lee _frame de un archivo csv
def read_to_csv(filename1,filename2):
    saccades = pd.read_csv (filename1)
    data = pd.read_csv (filename2)
    return (saccades,data)